We start by importing the libraries that will be required.

In [1]:
# library to handle requests
import requests

# library for data analsysis
import pandas as pd

# library to handle data in a vectorized manner
import numpy as np

# module which can convert addresses into latitude and longitude values
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# required for tranforming json files into a pandas dataframe library
from pandas.io.json import json_normalize

# library for plotting maps
!conda install -c conda-forge folium=0.5.0 --yes
import folium

# Prints that everything has been installed and imported once this cell has finished running
print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

In [2]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: ENKWV0CPFT0GLVSMPEYNLKSPECIYIUUSF1RPIAHPCLGRZK1M
CLIENT_SECRET:O2WTPCQNQ0YOTYDAUJFZGAKMJAQPQPECWZXHWHRIQG1WGGCH


The first thing we need to do for our Foursquare query is to find the latitude and longitude coordinates for our first venue.

In [3]:
# Finding the latitude and longitude for Madison Square Gardens
address = 'Madison Square Garden, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# This will print the latititude and longitude once found
print(latitude, longitude)

40.7505247 -73.99355027800776


We then need to set up our variables to get the information that we want to find.
In this case we are looking for bars that are within 1.5km of either Madison Square Gardens or Yankee Stadium.

In [4]:
# My desired search variable will be for bars around the main site
search_query = 'Bar'

# The radius variable has been set to 1.5km
radius = 1500

# Once this has run we will get confirmation that it has been processed
print(search_query + ' .... OK!')

Bar .... OK!


The url variable needs to be set up which includes our client ID and password, as well as the variables for our query which we have previously set up. This lets foursquare determine the user making the query and which results to return.

In [5]:
# This is setting the url to include my desired query variables in the Foursquare search
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

'https://api.foursquare.com/v2/venues/search?client_id=ENKWV0CPFT0GLVSMPEYNLKSPECIYIUUSF1RPIAHPCLGRZK1M&client_secret=O2WTPCQNQ0YOTYDAUJFZGAKMJAQPQPECWZXHWHRIQG1WGGCH&ll=40.7505247,-73.99355027800776&v=20180604&query=Bar&radius=1500&limit=500'

In [6]:
# This cell wiil return the results from the Foursquare query search
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f0ec32c01a4110ebc115289'},
 'response': {'venues': [{'id': '5a01dbb675a6ea7cadc95caf',
    'name': 'Pret Bar',
    'location': {'lat': 40.75057,
     'lng': -73.99352,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.75057,
       'lng': -73.99352}],
     'distance': 5,
     'postalCode': '10121',
     'cc': 'US',
     'neighborhood': 'Chelsea',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['New York, NY 10121', 'United States']},
    'categories': [{'id': '4bf58dd8d48988d116941735',
      'name': 'Bar',
      'pluralName': 'Bars',
      'shortName': 'Bar',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1594803215',
    'hasPerk': False},
   {'id': '4b6c9d90f964a520c5462ce3',
    'name': "Kabooz's Bar and Grill",
    'location': {'address': '2 Penn Plz',
     'crossStreet':

We can now take the results returned to us by foursquare and put them into a dataframe that is more pleasing to the eye. The top 5 results are looked at in the new dataframe for verification purposes.

In [7]:
# This assigns the relevant part of the received JSON file to venues
venues = results['response']['venues']

# We tranform the received venues around Madison Square Garden into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,5a01dbb675a6ea7cadc95caf,NaN,...,"[New York, NY 10121, United States]","[{'label': 'display', 'lat': 40.75057, 'lng': ...",40.750570,-73.993520,Chelsea,10121,NY,Pret Bar,v-1594803215,NaN
1,"[{'id': '4bf58dd8d48988d157941735', 'name': 'N...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b6c9d90f964a520c5462ce3,2 Penn Plz,...,"[2 Penn Plz (at Penn Station), New York, NY 10...","[{'label': 'display', 'lat': 40.75040735494259...",40.750407,-73.993081,NaN,10121,NY,Kabooz's Bar and Grill,v-1594803215,NaN
2,"[{'id': '4bf58dd8d48988d118941735', 'name': 'D...",NaN,NaN,NaN,NaN,NaN,NaN,False,4a71b57cf964a52067d91fe3,389 8th Ave,...,"[389 8th Ave (btw 29th & 30th Sts), New York, ...","[{'label': 'display', 'lat': 40.74929668498292...",40.749297,-73.995524,NaN,10001,NY,Walter's Bar,v-1594803215,43232524
3,"[{'id': '50be8ee891d4fa8dcc7199a7', 'name': 'M...",NaN,NaN,NaN,NaN,NaN,NaN,False,5c88110ed69ed00039580e96,250 7th Ave,...,"[250 7th Ave (West 25th Street), New York, NY ...","[{'label': 'display', 'lat': 40.745335, 'lng':...",40.745335,-73.994910,Chelsea,10001,NY,WFM Coffee Bar,v-1594803215,NaN
4,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,41059b00f964a520810b1fe3,407 8th Ave,...,[407 8th Ave (8th Avenue between W 30th & W 31...,"[{'label': 'display', 'lat': 40.74997672878124...",40.749977,-73.995102,NaN,10001,NY,Tempest Bar,v-1594803215,39892231


The dataframe shown above obviously needs some tidying up. We select the columns that are of interest or use to us and use a function to show the categories of the venues, which we filter clean.

In [8]:
# We only want to keep columns that include a venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# This function will extract the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# We can then filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# Finally we clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

# Show the first 5 rows of the dataframe to check the changes made
dataframe_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Pret Bar,Bar,NaN,US,New York,United States,NaN,5,"[New York, NY 10121, United States]","[{'label': 'display', 'lat': 40.75057, 'lng': ...",40.750570,-73.993520,Chelsea,10121,NY,5a01dbb675a6ea7cadc95caf
1,Kabooz's Bar and Grill,New American Restaurant,2 Penn Plz,US,New York,United States,at Penn Station,41,"[2 Penn Plz (at Penn Station), New York, NY 10...","[{'label': 'display', 'lat': 40.75040735494259...",40.750407,-73.993081,NaN,10121,NY,4b6c9d90f964a520c5462ce3
2,Walter's Bar,Dive Bar,389 8th Ave,US,New York,United States,btw 29th & 30th Sts,215,"[389 8th Ave (btw 29th & 30th Sts), New York, ...","[{'label': 'display', 'lat': 40.74929668498292...",40.749297,-73.995524,NaN,10001,NY,4a71b57cf964a52067d91fe3
3,WFM Coffee Bar,Market,250 7th Ave,US,New York,United States,West 25th Street,588,"[250 7th Ave (West 25th Street), New York, NY ...","[{'label': 'display', 'lat': 40.745335, 'lng':...",40.745335,-73.994910,Chelsea,10001,NY,5c88110ed69ed00039580e96
4,Tempest Bar,Pub,407 8th Ave,US,New York,United States,8th Avenue between W 30th & W 31st,144,[407 8th Ave (8th Avenue between W 30th & W 31...,"[{'label': 'display', 'lat': 40.74997672878124...",40.749977,-73.995102,NaN,10001,NY,41059b00f964a520810b1fe3


We can now see a list of all the relevant venues from our query and confirm that this is what we were looking for.

In [9]:
dataframe_filtered.name

0                                              Pret Bar
1                                Kabooz's Bar and Grill
2                                          Walter's Bar
3                                        WFM Coffee Bar
4                                           Tempest Bar
5                            Niles NYC Bar & Restaurant
6                                  The Club Bar & Grill
7                                     Muses Bar Karaoke
8                               Scallywag's Bar & Grill
9                                          Hibernia Bar
10                                Rowland’s Bar & Grill
11                                 Dalton's Bar & Grill
12                                           Bar Feroce
13                             Harrington's Bar & Grill
14                            Ayza Wine & Chocolate Bar
15                                  36 West Bar & Grill
16                    T.S. Ma Asian Cuisine & Sushi Bar
17                                    Ritz Bar &

In [17]:
dataframe_filtered.shape

(50, 16)

The object is to find which venue out of Madison Square Gardens or Yankee Stadium would be a more suitable location for a new sports bar with regards to competition in the area. We therefore need to bring in the second venue to the equation and find the coordinates for Yankee Stadium.
We now go through all of the processes for Yankee Stadium that we did previously for Madison Square Garden.

In [10]:
address_ys = 'Yankee Stadium, NY'

geolocator_ys = Nominatim(user_agent="foursquare_agent")
location_ys = geolocator_ys.geocode(address_ys)
latitude_ys = location_ys.latitude
longitude_ys = location_ys.longitude
print(latitude_ys, longitude_ys)

40.82958275 -73.92652118491901


In [11]:
url_ys = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_ys, longitude_ys, VERSION, search_query, radius, LIMIT)
url_ys

'https://api.foursquare.com/v2/venues/search?client_id=ENKWV0CPFT0GLVSMPEYNLKSPECIYIUUSF1RPIAHPCLGRZK1M&client_secret=O2WTPCQNQ0YOTYDAUJFZGAKMJAQPQPECWZXHWHRIQG1WGGCH&ll=40.82958275,-73.92652118491901&v=20180604&query=Bar&radius=1500&limit=500'

In [12]:
results_ys = requests.get(url_ys).json()
results_ys

{'meta': {'code': 200, 'requestId': '5f0ec3fe5cb8b40ce017b0d3'},
 'response': {'venues': [{'id': '4bf1ec69324cc9b672a5cc92',
    'name': "Tommy Bahama's Bar",
    'location': {'address': 'Yankee Stadium',
     'crossStreet': 'Gate 4, upstairs from Great Hall',
     'lat': 40.82904517052665,
     'lng': -73.92766149280284,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.82904517052665,
       'lng': -73.92766149280284}],
     'distance': 113,
     'postalCode': '10452',
     'cc': 'US',
     'city': 'Bronx',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['Yankee Stadium (Gate 4, upstairs from Great Hall)',
      'Bronx, NY 10452',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d11d941735',
      'name': 'Sports Bar',
      'pluralName': 'Sports Bars',
      'shortName': 'Sports Bar',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/sportsbar_',
       'suffix': '.png'},
      'primary': True}],
  

In [13]:
# assign relevant part of JSON to venues
venues_ys = results_ys['response']['venues']

# tranform venues into a dataframe
dataframe_ys = json_normalize(venues_ys)
dataframe_ys.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d11d941735', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,4bf1ec69324cc9b672a5cc92,Yankee Stadium,...,"[Yankee Stadium (Gate 4, upstairs from Great H...","[{'label': 'display', 'lat': 40.82904517052665...",40.829045,-73.927661,NaN,10452,NY,Tommy Bahama's Bar,v-1594803215,NaN
1,"[{'id': '4bf58dd8d48988d11d941735', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,4ba508aff964a5206dd338e3,Yankee Stadium,...,"[Yankee Stadium (Center Field), Bronx, NY 1046...",NaN,40.829778,-73.925738,NaN,10461,NY,Mohegan Sun Sports Bar,v-1594803215,NaN
2,"[{'id': '4bf58dd8d48988d117951735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,4bdb7d163904a59344c24a9e,Yankee Stadium,...,"[Yankee Stadium, Bronx, NY 10451-2100, United ...","[{'label': 'display', 'lat': 40.82965552618926...",40.829656,-73.926261,NaN,10451-2100,NY,Dylan's Candy Bar,v-1594803215,NaN
3,"[{'id': '4bf58dd8d48988d11d941735', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,3fd66200f964a520e1f01ee3,856 River Ave,...,"[856 River Ave (at E 161st St), Bronx, NY 1045...","[{'label': 'display', 'lat': 40.8272129, 'lng'...",40.827213,-73.925944,NaN,10451,NY,Billy's Sports Bar,v-1594803215,106032625
4,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",912823,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/applebees-grill-...,False,4ba0fb91f964a5200a8c37e3,610 Exterior St,...,"[610 Exterior St, Bronx, NY 10451, United States]","[{'label': 'display', 'lat': 40.8233799, 'lng'...",40.823380,-73.930627,NaN,10451,NY,Applebee's Grill + Bar,v-1594803215,NaN


In [14]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_ys = ['name', 'categories'] + [col for col in dataframe_ys.columns if col.startswith('location.')] + ['id']
dataframe_filtered_ys = dataframe_ys.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_ys['categories'] = dataframe_filtered_ys.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_ys.columns = [column.split('.')[-1] for column in dataframe_filtered_ys.columns]

dataframe_filtered_ys.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Tommy Bahama's Bar,Sports Bar,Yankee Stadium,US,Bronx,United States,"Gate 4, upstairs from Great Hall",113,"[Yankee Stadium (Gate 4, upstairs from Great H...","[{'label': 'display', 'lat': 40.82904517052665...",40.829045,-73.927661,NaN,10452,NY,4bf1ec69324cc9b672a5cc92
1,Mohegan Sun Sports Bar,Sports Bar,Yankee Stadium,US,Bronx,United States,Center Field,69,"[Yankee Stadium (Center Field), Bronx, NY 1046...",NaN,40.829778,-73.925738,NaN,10461,NY,4ba508aff964a5206dd338e3
2,Dylan's Candy Bar,Candy Store,Yankee Stadium,US,Bronx,United States,NaN,23,"[Yankee Stadium, Bronx, NY 10451-2100, United ...","[{'label': 'display', 'lat': 40.82965552618926...",40.829656,-73.926261,NaN,10451-2100,NY,4bdb7d163904a59344c24a9e
3,Billy's Sports Bar,Sports Bar,856 River Ave,US,Bronx,United States,at E 161st St,268,"[856 River Ave (at E 161st St), Bronx, NY 1045...","[{'label': 'display', 'lat': 40.8272129, 'lng'...",40.827213,-73.925944,NaN,10451,NY,3fd66200f964a520e1f01ee3
4,Applebee's Grill + Bar,American Restaurant,610 Exterior St,US,Bronx,United States,NaN,772,"[610 Exterior St, Bronx, NY 10451, United States]","[{'label': 'display', 'lat': 40.8233799, 'lng'...",40.823380,-73.930627,NaN,10451,NY,4ba0fb91f964a5200a8c37e3


In [16]:
dataframe_filtered_ys.name

0                        Tommy Bahama's Bar
1                    Mohegan Sun Sports Bar
2                         Dylan's Candy Bar
3                        Billy's Sports Bar
4                    Applebee's Grill + Bar
5        El Caldero Restaurant & Sports Bar
6                  Blue Point Bleachers Bar
7                     Glacken's Bar & Grill
8         Sal Y Pimienta (Bar & Restaurant)
9        Sam's Soul Food Restaurant and Bar
10                          Yankee Twin Bar
11                               Bareburger
12                               Secret Bar
13                          Underground Bar
14                        Stan's Sports Bar
15                 The patio grille and Bar
16                       Vistoso Beauty Bar
17         Blue Mountain Heights Cafe & Bar
18                               Munkee Bar
19                      Emergency Snack Bar
20                         Uptown Juice Bar
21           Dugout Sports Bar & Restaurant
22                              

In [18]:
dataframe_filtered_ys.shape

(50, 16)

Now that we have obtained all of the data that we wanted to see, we can go ahead and plot the points on the map using folium.
We start by adding the location of Madison Square Garden, (which is shown as a red circle) followed by the bars that would be within 1.5km of it (shown by blue circles).
Yankee Stadium is then added (shown as a black circle) and for clarity, the bars within 1.5km of Yankee Stadium are shown by green circles.

In [20]:
# Start by generating a map centred around Madison Square Garden
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# A red circle marker is added to represent the location of Madison Square Garden
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Madison Square Garden',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)


# We then add the bars asscoiated with MSG as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
    
# We then add Yankee Stadium and show its location as a balck circle
folium.features.CircleMarker(
    [latitude_ys, longitude_ys],
    radius=10,
    color='black',
    popup='Yankee Stadium',
    fill = True,
    fill_color = 'black',
    fill_opacity = 0.6
).add_to(venues_map)

# Finally we add the bars associated with Yankee Stadium as green circle markers
for lat, lng, label in zip(dataframe_filtered_ys.lat, dataframe_filtered_ys.lng, dataframe_filtered_ys.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(venues_map)

# We can now display the map
venues_map

From our map we can see that the Madison Square Garden bars are very tightly clustered around the main location. The bars around Yankee Stadium are a lot more spread out in comparison. The main find here though, is that there is a big 'hollow' area without and bars on the east side of Yankee Stadium. This therefore might be a good area to start looking for the placement of the new bar.

In [22]:
dataframe_filtered_ys.dtypes

name                 object
categories           object
address              object
cc                   object
city                 object
country              object
crossStreet          object
distance              int64
formattedAddress     object
labeledLatLngs       object
lat                 float64
lng                 float64
neighborhood         object
postalCode           object
state                object
id                   object
dtype: object

Now that we have decided that Yankee Stadium would be the more likely place to set up a new bar we can look at the clustering of the bars around the stadium.
We import sklearn to run the kmeans and allocate the bars into clusters.

In [40]:
from sklearn.cluster import KMeans
ysclusters = 4
ys_clustering = dataframe_filtered_ys[["distance", "lat","lng"]]
yskmeans = KMeans(n_clusters = ysclusters, random_state=0).fit(ys_clustering)
yskmeans.labels_[0:10]

array([1, 1, 1, 1, 0, 0, 1, 0, 2, 0], dtype=int32)

Having allocated a cluster to each bar, we can add the cluster back into the dataframe as their own column.

In [41]:
ys_combine = dataframe_filtered_ys.copy()
ys_combine["ClusterLabels"] = yskmeans.labels_
ys_combine.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id,ClusterLabels
0,Tommy Bahama's Bar,Sports Bar,Yankee Stadium,US,Bronx,United States,"Gate 4, upstairs from Great Hall",113,"[Yankee Stadium (Gate 4, upstairs from Great H...","[{'label': 'display', 'lat': 40.82904517052665...",40.829045,-73.927661,NaN,10452,NY,4bf1ec69324cc9b672a5cc92,1
1,Mohegan Sun Sports Bar,Sports Bar,Yankee Stadium,US,Bronx,United States,Center Field,69,"[Yankee Stadium (Center Field), Bronx, NY 1046...",NaN,40.829778,-73.925738,NaN,10461,NY,4ba508aff964a5206dd338e3,1
2,Dylan's Candy Bar,Candy Store,Yankee Stadium,US,Bronx,United States,NaN,23,"[Yankee Stadium, Bronx, NY 10451-2100, United ...","[{'label': 'display', 'lat': 40.82965552618926...",40.829656,-73.926261,NaN,10451-2100,NY,4bdb7d163904a59344c24a9e,1
3,Billy's Sports Bar,Sports Bar,856 River Ave,US,Bronx,United States,at E 161st St,268,"[856 River Ave (at E 161st St), Bronx, NY 1045...","[{'label': 'display', 'lat': 40.8272129, 'lng'...",40.827213,-73.925944,NaN,10451,NY,3fd66200f964a520e1f01ee3,1
4,Applebee's Grill + Bar,American Restaurant,610 Exterior St,US,Bronx,United States,NaN,772,"[610 Exterior St, Bronx, NY 10451, United States]","[{'label': 'display', 'lat': 40.8233799, 'lng'...",40.823380,-73.930627,NaN,10451,NY,4ba0fb91f964a5200a8c37e3,0


We can now sort the dataframe so that they are shown in order of their clusters. This gives us a clearer view of which bars have been put into the same cluster.

In [42]:
ys_combine.sort_values(["ClusterLabels"], inplace = True)
ys_combine

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id,ClusterLabels
40,G Bar,Karaoke Bar,156 E 150th St,US,Bronx,United States,Cedar Lane,1140,"[156 E 150th St (Cedar Lane), Bronx, NY, Unite...","[{'label': 'display', 'lat': 40.8193470800279,...",40.819347,-73.927066,NaN,NaN,NY,4c831b9adc018cfa9a53d66c,0
28,Downtown Bronx Bar & Cafe,Cocktail Bar,141 E 149th St,US,Bronx,United States,NaN,1193,"[141 E 149th St, Bronx, NY 10451, United States]","[{'label': 'display', 'lat': 40.81892013549805...",40.818920,-73.928040,NaN,10451,NY,4e4cf987bd413c4cc66daca4,0
4,Applebee's Grill + Bar,American Restaurant,610 Exterior St,US,Bronx,United States,NaN,772,"[610 Exterior St, Bronx, NY 10451, United States]","[{'label': 'display', 'lat': 40.8233799, 'lng'...",40.823380,-73.930627,NaN,10451,NY,4ba0fb91f964a5200a8c37e3,0
5,El Caldero Restaurant & Sports Bar,Latin American Restaurant,1267 Jerome Ave,US,Bronx,United States,NaN,1007,"[1267 Jerome Ave, Bronx, NY 10452, United States]","[{'label': 'display', 'lat': 40.837514, 'lng':...",40.837514,-73.920771,NaN,10452,NY,530643b7498e2069a0416c3f,0
27,Downtown Bronx Bar and Cafe,Coffee Shop,141 E 149th St,US,Bronx,United States,NaN,1185,"[141 E 149th St, Bronx, NY 10451, United States]","[{'label': 'display', 'lat': 40.81899600000000...",40.818996,-73.928019,NaN,10451,NY,4f32535619836c91c7cca951,0
7,Glacken's Bar & Grill,Bar,135 E 149th St,US,Bronx,United States,At Walton Avenue,1194,"[135 E 149th St (At Walton Avenue), Bronx, NY ...","[{'label': 'display', 'lat': 40.81898784783627...",40.818988,-73.928765,NaN,10451,NY,4c152bd31b5cef3b3240eec4,0
23,Piano Bar 1285 Shakesperar Ave,Nightclub,NaN,US,Bronx,United States,NaN,1077,"[Bronx, NY, United States]","[{'label': 'display', 'lat': 40.838722, 'lng':...",40.838722,-73.922310,NaN,NaN,NY,5227f1c711d21963cfb98765,0
9,Sam's Soul Food Restaurant and Bar,Southern / Soul Food Restaurant,596 Grand Concourse,US,Bronx,United States,btw 150th and 151st street,1086,[596 Grand Concourse (btw 150th and 151st stre...,"[{'label': 'display', 'lat': 40.81982606165913...",40.819826,-73.926618,NaN,10451,NY,4ca8dd85965c9c74c9dbc9fa,0
22,000 bar,None,Yankee stadium,US,Bronx,United States,NaN,985,"[Yankee stadium, Bronx, NY 10452, United States]","[{'label': 'display', 'lat': 40.83841100049279...",40.838411,-73.925721,NaN,10452,NY,4e67d22ffa76f38efb48e7a4,0
21,Dugout Sports Bar & Restaurant,Latin American Restaurant,120 E 151ST,US,Bronx,United States,@ Gerard Ave,864,"[120 E 151ST (@ Gerard Ave), Bronx, NY 10451, ...","[{'label': 'display', 'lat': 40.82192871964385...",40.821929,-73.928225,NaN,10451,NY,50113cbde4b0c79e46db2053,0


We now import extra libraries that give as access to the colour tools wanted to allocate colours to each of the clusters to make them show up on the map clearly in their cluster groups.

In [58]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colours

Finally we can show the cluster groups on the map, each having been allocated their own colour.

In [60]:
ys_map_clusters = folium.Map(location=[latitude_ys, longitude_ys], zoom_start=14)

x=np.arange(ysclusters)
ys= [i+x+(i*x)**2 for i in range(ysclusters)]
colours_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colours.rgb2hex(i) for i in colours_array]

markers_colors = []
for lat, lng, label, cluster in zip(ys_combine.lat, ys_combine.lng, ys_combine.categories, ys_combine.ClusterLabels):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color=rainbow[cluster - 1],
        popup=label,
        fill = True,
        fill_color=rainbow[cluster -1],
        fill_opacity=0.6).add_to(ys_map_clusters)

ys_map_clusters

Looking at the map we can see that the purple markers are very close to the stadium and that there are quite a lot of them close together. The red markers are the next closest and confirm the earlier thoughts that there isn't a bar that is in this distance range to the east of the stadium. The turqouise markers have a good dispersal around the outer distance range but again show a gap in the easterly direction. We can also gather from the map that a southerly direction would be one to avoid for the new bar due to the comparatively large cluster of bars there.